In [ ]:
!pip install -qq transformers[torch]
!pip install -qq sentencepiece
!pip install -qq datasets

In [ ]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import torch, random
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForLanguageModeling
from typing import Dict, List
from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names, load_from_disk, load_metric
#from torchmetrics import SacreBLEUScore
from tqdm.autonotebook import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
special_tokens = {'additional_special_tokens': ['<cls>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens)

model_three = MT5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CS685-Project/model checkpoints/objective_three/model_checkpointed').to(device)
model_one = MT5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CS685-Project/model checkpoints/objective_one/model_checkpointed').to(device)

In [ ]:
def translation_inference(input_sentences, tgt_lang, model, tokenizer, device):

  for i in range(len(input_sentences)):
    input_sentences[i] = tgt_lang + ' ' + input_sentences[i]

  print(input_sentences)

  inputs_tokenized = tokenizer(input_sentences, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
  outputs = model.generate(inputs_tokenized).to(device)

  output_sentences = []
  for op in outputs:
    output_sentences.append(tokenizer.decode(op, skip_special_tokens=True))

    #print(op.shape)


  return output_sentences



In [ ]:
sentences = ["Hey how are you?", "I am very hungry, lets have lunch", "Don't you think that it is a conspiracy?"]


output_sentences  = translation_inference(input_sentences = sentences, tgt_lang = 'hi', model = model_one, tokenizer = tokenizer, device = device)

print(output_sentences)

In [ ]:
sentences = ["मैं आपसे मिलने के लिए उत्साहित हूँ।", "आज का दिन बहुत खुशनुमा है। ", "ती पुस्तकांच्या प्रेमात पडली", "चित्रापटात जे घडते ते खरोखर नास्त"]

output_sentences  = translation_inference(input_sentences = sentences, tgt_lang = 'en ', model = model_three, tokenizer = tokenizer, device = device)

print(output_sentences)

In [ ]:
def calculate_style_embedding(sentences_list , style, model, tokenizer, device):

  inputs = tokenizer(sentences_list, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
  
  input_ids, input_masks = inputs.input_ids.to(device), inputs.attention_mask.to(device)


  style_batch = model.encoder(input_ids, attention_mask=input_masks).last_hidden_state[:, 0, :]
  #print(style_batch.shape)
  style_embedding_avg = torch.mean(style_batch, dim = 0)
  #print(style_embedding_avg.shape)

  return style_embedding_avg



def style_transfer_inference(input_sentence, sentences1_list, sentences2_list, style1, style2, src_lang, lambda_param, model, tokenizer, device):


  style_1_embedding = calculate_style_embedding(sentences1_list, style1, model, tokenizer, device)
  style_2_embedding = calculate_style_embedding(sentences2_list, style2, model, tokenizer, device)

  # input_sentence = 'en ' + input_sentence
  # inputs_tokenized = tokenizer(input_sentence, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
  # input_sentence_en = model.generate(inputs_tokenized).to(device) 
  en_input_sentence = translation_inference(input_sentences = [input_sentence], tgt_lang = 'en ', model = model, tokenizer = tokenizer, device = device)[0]

  #en_input_sentence = src_lang + ' ' + en_input_sentence

  modified_input_sentence = translation_inference(input_sentences = [en_input_sentence], tgt_lang = src_lang, model = model_one, tokenizer = tokenizer, device = device)[0]

  inputs_tokenized = tokenizer(modified_input_sentence, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

  input_ids = inputs_tokenized.input_ids.to(device)
  input_masks = inputs_tokenized.attention_mask.to(device)

  input_encoding = model.encoder(input_ids).last_hidden_state + lambda_param * (style_1_embedding - style_2_embedding)

  decoder_outputs = model.decoder(input_ids = input_ids, attention_mask = input_masks, encoder_hidden_states = input_encoding)

  model_outputs = model.lm_head(decoder_outputs.last_hidden_state).squeeze()

  #print("model_outputs.shape: ", model_outputs.shape)

  model_outputs_argmax = torch.argmax(model_outputs, dim = 1)

  output_sentence = tokenizer.decode(model_outputs_argmax, skip_special_tokens=True)

  return output_sentence



In [ ]:
# informal -> formal Hindi
sentences1_list = ["तुझे मेरी मदद चाहिये क्या ?",
"क्या तेरा अकस्मात हो गया ?",
"उसने मुझे बोला की उधर प्राणवायु बहुत कम था | ",
"इधर की व्यवस्था ठीक-ठाक है | ",
"अपन पद का फायदा नहीं उठा सकते | ",
"तुजे आज भोजन में क्या खाना है ?",
"तुजे घर ढूंढने में कोई दिक्कत नहीं हुई ना ?"]

sentences2_list = ["क्या आपको मेरी सहायता चाहिये?",
"क्या आपका अकस्मात हो गया था ?",
"उन्होंने मुझे बोला की उधर प्राणवायु बहुत मर्यादित था |",
"यहाँ की व्यवस्था सामान्य है |",
"हम अपने पद का दुरुपयोग नहीं कर सकते |" ,
"तुम आज भोजन में क्या खाना पसंद करोगे ?,"
"आपको घर ढूंढने में कोई परेशानी तो नहीं हुई ना ?"]

input_sentence = "पापा कार्यालय में जरुरी काम करते है"

output_sentence = style_transfer_inference(input_sentence, sentences1_list, sentences2_list, 'informal', 'informal', 'hi', 1, model_three, tokenizer, device)

print(output_sentence)

In [ ]:
# pure -> impure Hindi
sentences1_list = ["मैं अखबार पढ रहा हूँ |",
"आज विद्यालय जाते समय मैंने एक बहुत ही भयानक अकस्मात देखा |",
"नमस्कार | में आप सभी का हार्दिक स्वागत करता हूँ और आशा करता हूँ की आपको आने में कोई परेशानी नहीं हुई |",
"मुंबई और बैंगलोर जैसे बड़े शहरों में बहुत ज्यादा यातायात होता है।",
"माफ़ कीजिए, मैं आपको ठीक से समझ नहीं पाया था।",
"मेरे पिताजी कार्यालय में एक महत्वपूर्ण पद पर काम करते हैं।",
"छात्रों ने विद्यालय में नाटक में अभिनय किया और सबको बहुत हसाया।"]

sentences2_list = ["मैं न्यूज़पेपर पढ रहा हूँ |",
"आज स्कूल जाते समय मैंने एक बहुत ही भयानक एक्सीडेंट देखा |",
"हेल्लो  | में आप सभी का हार्दिक स्वागत करता हूँ और आशा करता हूँ की आपको आने में कोई परेशानी नहीं हुई |",
"मुंबई और बैंगलोर जैसे बड़े शहरों में बहुत ज्यादा ट्र्रैफिक  होता है।",
"सॉरी , मैं आपको ठीक से समझ नहीं पाया था।" ,
"मेरे डैडी ऑफ़िस में एक इम्पोर्टेंट पोज़ीशन पर काम करते हैं।"
"छात्रों ने स्कूल में नाटक में एक्टिंग की और सबको बहुत हसाया।"]

input_sentence = "आज शाम मैं वॉक पर गई थी और मैंने गार्डन में पांच मोर देखे "

output_sentence = style_transfer_inference(input_sentence, sentences2_list, sentences1_list, 'impure', 'pure', 'hi', 1, model_three, tokenizer, device)

print(output_sentence)

In [ ]:
# informal -> formal marathi
sentences1_list = ["सरकारमधील माहिती तंत्रज्ञान प्रशिक्षणासाठी समन्वय करणारी संस्था बनशील ",
"अर्थसंकल्पीय अधिवेशन दोन टप्प्यात केल जात आहे. ?",
"कोरोना विषाणूशी लढण्यासाठी सरकारच्या तयारीचा उच्चस्तरीय आढावा आहे.",
"इराणने पुन्हा अमेरिकेला धमकी दिली आहे.",
"आणि तुझ्या प्रभूबद्दल तुझ्या वाईट विचारांनी तुझा नाश केला आणि तू नुकसानीत होता.",
"मी अधिक दिवसांची शपथ घेतो जेव्हा ते खूप उज्ज्वल असते.",
"जैमिनी सूत्रानुसार गीतेला साम म्हणतात."]

sentences2_list = ["सरकारमधील माहिती तंत्रज्ञान प्रशिक्षणासाठी समन्वय करणारी संस्था बनाल ",
"अर्थसंकल्पीय अधिवेशन दोन टप्प्यात आयोजित करत आहे.",
"सरकार कोरोना विषाणूला रोखण्यासाठी सज्जतेचा उच्चस्तरीय आढावा घेत आहे.",
"इराण पुन्हा अमेरिकेशी संघर्षाच्या मार्गावर.",
"आणि तुमचा हा चुकीचा विचार जो तुम्ही तुमच्या प्रभूबद्दल बाळगत होता, त्यामुळे तुमचा सर्वनाश झाला, शेवटी तुम्ही नुकसानीत होता." ,
"आणि ज्या दिवशी ते उजळेल."
"जैमिनी सूत्रानुसार - गीतकाला सामन म्हणतात."]

input_sentence = "आपण कशाबद्दल प्रार्थना करू शकतो?"

output_sentence = style_transfer_inference(input_sentence, sentences1_list, sentences2_list, 'pure', 'impure', 'mr', 1, model_three, tokenizer, device)

print(output_sentence)

In [ ]:
# pure -> impure marathi
sentences1_list = ["पॅनवर हाल्फ भाजून मग फ्राय केल्याने एक्साक्टली काय होते? दाळ पक्वान मधले पक्वान फ्राय करतानाही हीच टीप मिळालेली आहे पण एक्साक्टली उद्देश कळलं नाही.",
"राफेल काँट्रॅक्टच्या मुद्यांवर प्राईम मिनिस्टर नरेंद्र मोदींनी उत्तर द्यावं. भारत आणि फ्रान्स यांच्यात राफेल काँट्रॅक्टची प्रोसेस चालू होती.",
"माझे दहावीचे वेळेचे प्लॅन्स अभ्यासापेक्षा मोकळीकच जास्त आहे. किती वर्ड्ज टाकावे लागतात आता.",
"28 नोव्हेंबर ते 16 डिसेंबर पर्यंत हॉकीचे सामने चालू राहणार असून टोटल 16 देशांनी या स्पर्धेत भाग घेतला आहे. या स्पर्धेसाठी टीम्सची चार ग्रुप्स डिव्हिजन करण्यात आली आहे.",
"मला खूपदा हा प्रश्न विचारला जातो कि मी कुठले बुक्स युझ केले आहे. मी मला आवडलेली काही बुक्स इथे देतो. ही लिस्ट मी उपडेत करत राहीन.",
"परवा हॉस्पिटलमध्ये गेले होते. निघायच्या वेळेला वाट्च  पाहिले आणि लक्षात आले ते एकदम 'राईट टाईम' होते.",
"भुवनेश्वर येथील कलिंगा येथील स्टेडियममध्ये आजपासून हॉकी वर्ल्ड कपच्या सामन्यांना सुरुवात होणार आहे. तसेच भारतीय हॉकी टीम उद्या दक्षिण अफ्रिकेविरुद्ध हॉकी सामना खेळण्यास रेडी झाला आहे."]

sentences2_list = ["तव्यावर अर्धवट भाजून मग तळल्याने नेमके काय होते? दाल पक्वान मधले पक्वान तळतानाही हीच पूर्वसूचना मिळालेली आहे पण नेमके प्रयोजन कळले नाही.",
"राफेल कराराच्या मुद्यांवर पंतप्रधान नरेंद्र मोदींनी उत्तर द्यावं. भारत आणि फ्रान्स यांच्यात राफेल कराराची प्रक्रिया चालू होती.",
"माझे १० वी चे वेळेचे नियोजन आभ्यासापेक्षा मोकळीकच जास्त आहे. किती शब्द टाकावे लागतात आता.",
"२८ नोव्हेंबर ते १६ डिसेंबर पर्यंत हॉकीचे सामने चालू राहणार असून एकूण १६ देशांनी या स्पर्धेत भाग घेतला आहे. या स्पर्धेसाठी संघांची चार गटांत विभागणी करण्यात अली आहे.",
"मला बरेचदा हा प्रश्न विचारला जातो की मी कुठले पुस्तक वापरले आहे. मी मला आवडलेली काही पुस्तकं इथे देतो. ही यादी मी अद्यतन करत राहीन." ,
"पर्वा दवाखान्यात गेले होते. निघायच्या वेळा घड्याळ पहिले आणि लक्षात आले ते एक्दम बरोबर वेळ होते."
"भुवनेश्वर येथील कलिंगा येथील रिंगणमध्ये आजपासून हॉकी विश्वचषकच्या सामन्यांना सुरवात होणार आहे. तसेच भारतीय हॉकी संघ उद्या आफ्रिके विरुद्ध हॉकी सामना खेळण्यास सज्ज झाला आहे. "]

input_sentence = "विरोधी पक्ष नेते राधाकृष्ण विखे पाटील यांनी शिवसेनेची चांगलीच खिल्ली उडवली."

output_sentence = style_transfer_inference(input_sentence, sentences2_list, sentences1_list, 'pure', 'impure', 'mr', 1, model_three, tokenizer, device)

print(output_sentence)

In [ ]:
# sentencelist_one

def multi_attribute_style_transfer_inference(input_sentence, sentencelist_one, sentence_list_two, senetence_list_three, style1, style2, style3, src_lang, lambda_param, model, tokenizer, device):
    style_1_embedding = calculate_style_embedding(sentencelist_one, style1, model, tokenizer, device)
    style_2_embedding = calculate_style_embedding(sentence_list_two, style2, model, tokenizer, device)
    style_3_embedding = calculate_style_embedding(senetence_list_three, style3, model, tokenizer, device)

    en_input_sentence = translation_inference(input_sentences = [input_sentence], tgt_lang = 'en ', model = model, tokenizer = tokenizer, device = device)[0]

    modified_input_sentence = translation_inference(input_sentences = [en_input_sentence], tgt_lang = src_lang, model = model_one, tokenizer = tokenizer, device = device)[0]

    inputs_tokenized = tokenizer(modified_input_sentence, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    input_ids = inputs_tokenized.input_ids.to(device)
    input_masks = inputs_tokenized.attention_mask.to(device)

    input_encoding = model.encoder(input_ids).last_hidden_state + lambda_param * (style_1_embedding + style_2_embedding - style_3_embedding)
    decoder_outputs = model.decoder(input_ids = input_ids, attention_mask = input_masks, encoder_hidden_states = input_encoding)

    model_outputs = model.lm_head(decoder_outputs.last_hidden_state).squeeze()

    model_outputs_argmax = torch.argmax(model_outputs, dim = 1)

    output_sentence = tokenizer.decode(model_outputs_argmax, skip_special_tokens=True)

    return output_sentence



In [ ]:
#formal + pure -> informality + impurity 

sentencelist_one = ["क्या आपको मेरी सहायता चाहिये?",
"क्या आपका अकस्मात हो गया था ?",
"उन्होंने मुझे बोला की उधर प्राणवायु बहुत मर्यादित था |",
"यहाँ की व्यवस्था सामान्य है |",
"हम अपने पद का दुरुपयोग नहीं कर सकते | ",
"तुम आज भोजन में क्या खाना पसंद करोगे ?"]

sentencelist_two = ["तुझे मेरी मदद चाहिये क्या ?",
"क्या तेरा अकस्मात हो गया ?",
"उसने मुझे बोला की उधर प्राणवायु बहुत कम था |" ,
"इधर की व्यवस्था ठीक-ठाक है | ",
"अपन पद का फायदा नहीं उठा सकते |" ,
"तुजे आज भोजन में क्या खाना है ?" ]

sentencelist_three = ["मैं न्यूज़पेपर पढ रहा हूँ | ",
"गुड मॉर्निंग ! आप कैसे हो ?",
"थैंक यू  ! यह उपकार मैं कभी नहीं भूलूंगी |" ,
"आज का ब्रेकफास्ट बहुत ही स्वादिष्ट था |",
"मैं आज लंच में क्या बनाऊ ?",
"आज स्कूल जाते समय मैंने एक बहुत ही भयानक एक्सीडेंट देखा |" ]

input_sentence = "आप प्रयत्न अवश्य करना |    "

multi_attribute_style_transfer_inference(input_sentence, sentencelist_one, sentencelist_two, sentencelist_three, 'formal + pure', 'informal + pure ', 'impure', 'hi ' ,1, model_three, tokenizer, device)

In [ ]:

#formal + pure -> informality + impurity 

sentencelist_one = ["शासनाची मदत कधीच पोहोचत नाही.",
"मग पक्षी उडतील.",
"आपण कशाबद्दल प्रार्थना करू शकतो?",
"आता आपण आणखी काय करू शकतो ते पाहूया",
"आणि तू शेवटची कधी भेटली होतीस?"]

sentencelist_two = ["सरकारने कधीही मदत केली नाही.",
"मग ते धान्यासारखे उडू लागतील.",
"आपण काय करू शकतो?",
"आता आपण काय करू शकतो ते पाहूया.",
"आणि शेवटच्या वेळी कधी मिठी मारली होतीस?" ]

sentencelist_three = ["झारखंडचे मुख्यमंत्री रघुबर दास यांनी म्हंटले आहे की, त्यांचे सरकार भारतातील सर्वात मोठे वॉर मेमोरियल बनवीत आहे.",
"देशभरात सर्वदूर पावसाने हजेरी लावली आहे. अनेक पिकांना नेसेसरी होता त्या वेळी पाऊस झाल्याने उत्पादनवाढीची अपेक्षा आहे.",
"१५ दिवसांपासून पावसाचा ब्रेक पडल्याने शेतकऱ्यांच्या नजरा आभाळा कडे होत्या.",
"शुगर कमी झ्हाल्यानं चक्कर आली, असं युनियन मिनिस्टर नितीन गडकरी यांनी ट्विट करून सांगितलं आहे.",
"मुंबईत रिपोर्टर कॉन्फेरेंन्स घेऊन तिनं हे आरोप केले आहे. अभिनेता सलमान खान आणि त्याच्या भावांनी माझ्या वडिलांचा मर्डर केला, असा आणखी एक शॉकिंग आरोपही पूजाने केले आहे." ]

input_sentence = "असा वाटतं की वडिल तुमश्यावर प्रेम नाही करत. "

multi_attribute_style_transfer_inference(input_sentence, sentencelist_one, sentencelist_two, sentencelist_three, 'formal + pure', 'informal + pure ', 'impure', 'mr ' ,1, model_three, tokenizer, device)

In [ ]:
# sentences = ["mr What is the secret behind this", "mr We must keep him under watch for a couple of years.", "mr Is it not a conspiracy?"]
# #sentences2 = ["hi Hey there! How are you doing", "hi An apple a day keeps the doctor away", "hi Is it not a conspiracy?"]

# inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
# outputs = model.generate(inputs).to(device)

# #print(outputs)

# for op in outputs:
#   ans = tokenizer.decode(op, skip_special_tokens=True)
#   print(ans)

In [ ]:
# sentences1 = ["en साजिश तो नहीं?"]

# sentences = sentences1 + sentences2
# inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
# outputs = model.generate(inputs).to(device)

# #print(outputs)

# for op in outputs:
#   ans = tokenizer.decode(op, skip_special_tokens=True)
#   print(ans)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")
# special_tokens = {'additional_special_tokens': ['<cls>']}
# num_added_toks = tokenizer.add_special_tokens(special_tokens)
# model = MT5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/CS685-Project/model checkpoints/objective_three/model_checkpointed').to(device)

In [ ]:
# def calculate_style_embeddings(sentences_list , style, model, tokenizer, device):

#   inputs = tokenizer(sentences_list, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
  
#   input_ids, input_masks = inputs.input_ids.to(device), inputs.attention_mask.to(device)


#   style_batch = model.encoder(input_ids, attention_mask=input_masks).last_hidden_state[:, 0, :]
#   #print(style_batch.shape)
#   style_embedding_avg = torch.mean(style_batch, dim = 0)
#   #print(style_embedding_avg.shape)

#   return style_embedding_avg

In [ ]:
# def style_transfer_inference(input_sentence, sentences1_list, sentences2_list, style1, style2, model, tokenizer, device):


#   style_1_embedding = calculate_style_embeddings(sentences1_list, style1, model, tokenizer, device)
#   style_2_embedding = calculate_style_embeddings(sentences2_list, style2, model, tokenizer, device)

#   input_sentence = 'en ' + input_sentence
#   inputs_tokenized = tokenizer(input_sentence, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
#   input_sentence_en = model.generate(inputs_tokenized).to(device) 

  


#   input_en = mo

In [ ]:
avg = calculate_style_embeddings(['hey there', 'hey there'], 'informal', model, tokenizer, device)

In [ ]:
# sentences = ["mr What is the secret behind this", "mr We must keep him under watch for a couple of years.", "mr Is it not a conspiracy?"]
# #sentences2 = ["hi Hey there! How are you doing", "hi An apple a day keeps the doctor away", "hi Is it not a conspiracy?"]

# inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids.to(device)
# outputs = model.generate(inputs).to(device)

# #print(outputs)

# for op in outputs:
#   ans = tokenizer.decode(op, skip_special_tokens=True)
#   print(ans)